# TF-IDF with spaCy

Date: 2024/05/03

In [8]:
# ChatGPTへ「適当な文章をつくってください」の応答

text1 = '夕方に勉強していたら、遠くで犬が吠えているのが聞こえた。たぶん、散歩中の他の犬を怖がっているのだろう。'
text2 = '毎日、通勤時間に勉強する。勉強は自己への投資。'
text3 = '勉強しない上司は嫌い。'
text4 = '駅まで歩いている途中、散歩中の犬が私に向かって吠えた。'
texts = [text1, text2, text3, text4]

In [30]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load('ja_core_news_sm')

In [41]:
patterns = [[{"TEXT": "勉強"}],[{"TEXT": "犬"}]]
matcher = Matcher(nlp.vocab)
matcher.add("日常", patterns)

In [42]:
for doc in nlp.pipe(texts):
    print(f'[len {len(doc)}]', end=' ')
    for token in doc:
        print(token.text, end=' ')
    print()

[len 36] 夕方 に 勉強 し て い たら 、 遠く で 犬 が 吠え て いる の が 聞こえ た 。 たぶん 、 散歩 中 の 他 の 犬 を 怖 がっ て いる の だろう 。 
[len 15] 毎日 、 通勤 時間 に 勉強 する 。 勉強 は 自己 へ の 投資 。 
[len 7] 勉強 し ない 上司 は 嫌い 。 
[len 19] 駅 まで 歩い て いる 途中 、 散歩 中 の 犬 が 私 に 向かっ て 吠え た 。 


## TF

In [49]:
all_tf = []

for doc in nlp.pipe(texts):
    cnt = {'勉強': 0, '犬': 0}
    print(f'[len {len(doc)}]', end=' ')
    matches = matcher(doc, as_spans=True)
    #print(len(matches))
    for span in matches:
        #print(span.text, span.label_)
        cnt[span.text] += 1
    tf = {}
    for k,v in cnt.items():
        tf[k] = v/len(doc)
    all_tf.append(tf)

all_tf

[len 36] [len 15] [len 7] [len 19] 

[{'勉強': 0.027777777777777776, '犬': 0.05555555555555555},
 {'勉強': 0.13333333333333333, '犬': 0.0},
 {'勉強': 0.14285714285714285, '犬': 0.0},
 {'勉強': 0.0, '犬': 0.05263157894736842}]

## IDF

In [51]:
sent_cnt = {}
for tf in all_tf:
    for k,v in tf.items():
        if k not in sent_cnt:
            sent_cnt[k] = 0
        if v > 0:
            sent_cnt[k] += 1

sent_cnt

{'勉強': 3, '犬': 2}

In [55]:
import math

all_idf = {}
           
for k,v in sent_cnt.items():
    all_idf[k] = math.log(len(texts)/v)

all_idf

{'勉強': 0.28768207245178085, '犬': 0.6931471805599453}

## TF-IDF

In [56]:
all_tf_idf = []

for tf in all_tf:
    tf_idf = {}
    for k,v in tf.items():
        tf_idf[k] = v * all_idf[k]
    all_tf_idf.append(tf_idf)

all_tf_idf

[{'勉強': 0.007991168679216135, '犬': 0.03850817669777474},
 {'勉強': 0.03835760966023745, '犬': 0.0},
 {'勉強': 0.04109743892168297, '犬': 0.0},
 {'勉強': 0.0, '犬': 0.03648143055578659}]